In [1]:
import pandas as pd
import psycopg2
import os

# List of CSV files and their corresponding table names
csv_files = [
    ('./hrdata.csv', 'hrdata'), # Added payments.csv for specific handling
]

# Folder containing the CSV files
folder_path = 'C:/Users/moham/Desktop/Data Analysis/All_PowerBi Project/HR Analysis'
# df = pd.read_csv(file_path, encoding='ISO-8859-1')


# Define a function to map pandas data types to SQL types
def get_sql_type(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return 'INTEGER'
    elif pd.api.types.is_float_dtype(dtype):
        return 'REAL'
    elif pd.api.types.is_bool_dtype(dtype):
        return 'BOOLEAN'
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return 'TIMESTAMP'
    else:
        return 'TEXT'

try:
    # Connect to the PostgreSQL database
    conn = psycopg2.connect(
        host='localhost',
        user='postgres',
        password='mohammad',
        dbname='HR Analysis',
        port='5432'  # typically 5432 for PostgreSQL
    )
    cursor = conn.cursor()
    print("Connected to the database.")

    # Loop over CSV files and load each into PostgreSQL
    for csv_file, table_name in csv_files:
        file_path = os.path.join(folder_path, csv_file)

        # Read the CSV file into a pandas DataFrame
        df = pd.read_csv('./hrdata.csv')

        # Replace NaN with None to handle SQL NULL
        df = df.where(pd.notnull(df), None)
        
        # Clean column names
        df.columns = [col.replace(' ', '_').replace('-', '_').replace('.', '_') for col in df.columns]

        # Generate the CREATE TABLE statement with appropriate data types
        columns = ', '.join([f'"{col}" {get_sql_type(df[col].dtype)}' for col in df.columns])
        create_table_query = f'CREATE TABLE IF NOT EXISTS "{table_name}" ({columns})'
        cursor.execute(create_table_query)
        print(f"Table `{table_name}` created or already exists.")

        # Insert DataFrame data into the PostgreSQL table
        for _, row in df.iterrows():
            # Convert row to tuple and handle NaN/None explicitly
            values = tuple(None if pd.isna(x) else x for x in row)
            sql = f"INSERT INTO \"{table_name}\" ({', '.join(['"' + col + '"' for col in df.columns])}) VALUES ({', '.join(['%s'] * len(row))})"
            cursor.execute(sql, values)

        # Commit the transaction for the current CSV file
        conn.commit()
        print(f"Data from `{csv_file}` inserted successfully into `{table_name}`.")

except psycopg2.Error as err:
    print(f"Error: {err}")
finally:
    # Close the cursor and connection
    if cursor:
        cursor.close()
    if conn:
        conn.close()
    print("Database connection closed.")

C:\Users\moham\AppData\Local\Temp\ipykernel_10956\465980585.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Connected to the database.
Table `hrdata` created or already exists.
Data from `./hrdata.csv` inserted successfully into `hrdata`.
Database connection closed.


## All Columns

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
data = pd.read_csv("./hrdata.csv")
data.head(3)

,emp_no,gender,marital_status,age_band,age,department,education,education_field,job_role,business_travel,employee_count,attrition,attrition_label,job_satisfaction,active_employee
0,10001,Female,Single,35 - 44,41,Sales,Associates Degree,Life Sciences,Sales Executive,Travel_Rarely,1,Yes,Ex-Employees,4,0
1,10002,Male,Married,45 - 54,49,R&D,High School,Life Sciences,Research Scientist,Travel_Frequently,1,No,Current Employees,2,1
2,10003,Male,Single,35 - 44,37,R&D,Associates Degree,Other,Laboratory Technician,Travel_Rarely,1,Yes,Ex-Employees,3,0


In [8]:
data.columns

Index(['emp_no', 'gender', 'marital_status', 'age_band', 'age', 'department',
       'education', 'education_field', 'job_role', 'business_travel',
       'employee_count', 'attrition', 'attrition_label', 'job_satisfaction',
       'active_employee'],
      dtype='object')

In [10]:

for col in data.columns:
    print(col)

emp_no
gender
marital_status
age_band
age
department
education
education_field
job_role
business_travel
employee_count
attrition
attrition_label
job_satisfaction
active_employee
